In [7]:
import yfinance as yf
import datetime as dt
import numpy as np
from scipy.stats import norm

In [58]:
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
        
S = 100        
K = 150          # Strike
T = 365 / 365     # 30 days to expiry
r = 0.05         # Risk-free rate (5%)
sigma = 0.3

option_type = 'put'

price = black_scholes(S, K, T, r, sigma, option_type)
print(price.round(2))

44.74
